In [1]:
from __future__ import division, absolute_import

import sys
import random
import pickle

import pandas as pd
import numpy as np
import h5py
from plotnine import *
import seaborn as sns

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)

from src.Target import Target
from src.imbalance_functions import computing_random_baseline

np.random.seed(8)
random.seed(8)

Using TensorFlow backend.


In [2]:
protein_type = "GPCRs" #"kinases"
nfolds = 10

In [3]:
#Loading list of unique proteins 
with open("".join((absPath, "data/", protein_type, "/", protein_type, "_prots.pickle")), 'rb') as handle:
    unique_prots = pickle.load(handle)

In [4]:
strategies = ["resampling_before_clustering", "no_resampling", "resampling_after_clustering", "semi_resampling"]

ratios_df_completo = pd.read_csv("".join((absPath, "data/", protein_type, "/ratios_df_complete.csv")))

In [5]:
ratios_df_completo.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)

In [6]:
ratios_df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772 entries, 0 to 10771
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  10772 non-null  object 
 1   ratio_training           9902 non-null   float64
 2   ratio_test               10772 non-null  float64
 3   ratio_test_predicted     10772 non-null  float64
 4   acc                      10772 non-null  float64
 5   auroc                    8157 non-null   float64
 6   f1                       10772 non-null  float64
 7   balanced_acc             10772 non-null  float64
 8   mcc                      10772 non-null  float64
 9   strategy                 10772 non-null  object 
 10  fold                     10772 non-null  int64  
 11  Sequence                 10772 non-null  object 
 12  family                   10772 non-null  object 
 13  Uniprot ID               10772 non-null  object 
 14  n_interactions        

In [7]:
ratios_df_completo.head()

,DeepAffinity Protein ID,ratio_training,ratio_test,ratio_test_predicted,acc,auroc,f1,balanced_acc,mcc,strategy,fold,Sequence,family,Uniprot ID,n_interactions,len_seq
0,UYMO,0.477448,0.620536,0.575893,0.776786,0.859035,0.813433,0.774439,0.538935,resampling_before_clustering,0,MWNATPSEEPGFNLTLADLDWDASPGNDSLGDELLQLFPAPLLAGV...,GPCR,Q92847,1004,366
1,0BJE,0.487952,0.604651,0.627907,0.790698,0.841629,0.830189,0.776018,0.558389,resampling_before_clustering,0,MTPNSTGEVPSPIPKGALGLSLALASLIITANLLLALGIAWDRRLR...,GPCR,Q8TDU6,423,330
2,78TA,0.503030,0.500000,0.550000,0.950000,0.910000,0.952381,0.950000,0.904534,resampling_before_clustering,0,MMTPNSTELSAIPMGVLGLSLALASLIVIANLLLALGIALDRHLRS...,GPCR,Q80SS6,138,329
3,RJYX,0.435644,0.444444,0.777778,0.444444,0.350000,0.545455,0.475000,-0.059761,resampling_before_clustering,0,METNFSTPLNEYEEVSYESAGYTVLRILPLVVLGVTFVLGVLGNGL...,GPCR,P25090,112,351
4,95E2,0.525958,0.307190,0.385621,0.758170,0.856534,0.650943,0.751455,0.476653,resampling_before_clustering,0,MGPTSVPLVKAHRSSVSDYVNYDIIVRHYNYTGKLNISADKENSIK...,GPCR,P21453,2471,382


we should not use n_interactions because that the TOTAL NUMBER of interactions, but we are interested only in test!
we have to load te corresponding prediction on test file, check the shape and take this number as n_interactions

In [8]:
#TESTING

#strategy = "no_resampling"
#fold = 0
#pred_test_path = "".join((absPath, "data/", protein_type, "/", strategy, "/predictions/", str(fold), "/test.csv"))
#predtest_loaded = pd.read_csv(pred_test_path, index_col=False)
#predtest_loaded

In [9]:
#predtest_loaded.comp_ID.values.tolist()

In [ ]:
list_rdm = []
for strategy in strategies:
    for fold in range(nfolds):
        for prot in unique_prots:
            dict_prot = {}
            rdm_comps = computing_random_baseline(ratios_df_completo, strategy, prot, fold, protein_type)
            dict_prot["prot"] = prot
            dict_prot["strategy"] = strategy
            dict_prot["random_baseline"] = rdm_comps 
            dict_prot["fold"] = fold
            #dict_prot["comps"] = comps
            dict_prot["idx"] = range(len(rdm_comps))
            list_rdm.append(dict_prot)

/home/angela3/imbalance_pcm_benchmark/src/imbalance_functions.py:228: RuntimeWarning: Mean of empty slice.
  ratio_training = subdf.loc[subdf["DeepAffinity Protein ID"]==prot, "ratio_training"].dropna().values.mean()
/home/angela3/venv/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
random_baseline_df = pd.DataFrame(list_rdm)

In [ ]:
# list_rdm[5]

In [ ]:
random_baseline_df.info()

In [ ]:
random_baseline_df.head()

In [ ]:
list_cols = ["random_baseline", "idx"]
other_cols = list(set(random_baseline_df.columns) - set(list_cols))
exploded = [random_baseline_df[col].explode() for col in list_cols]
exploded_rdm_bsl = pd.DataFrame(dict(zip(list_cols, exploded)))
exploded_rdm_bsl = random_baseline_df[other_cols].merge(exploded_rdm_bsl, how="right", 
                                                        left_index=True, right_index=True)
exploded_rdm_bsl

In [ ]:
exploded_rdm_bsl.head()

In [ ]:
exploded_rdm_bsl.info()

In [ ]:
exploded_rdm_bsl.to_csv("".join((absPath, "/data/", protein_type, "/random_baseline.csv")))